In [1]:
from sqlalchemy import create_engine
from db_constants import Base
from sqlalchemy import func
import pandas as pd
from models import ParentPair, ChildPair
from sqlalchemy.orm import sessionmaker
from sqlalchemy import String
from etl.data_loader import DataLoader


engine = create_engine('sqlite:///Cryptos.db', echo=False)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_2354/249799025.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
# print(len(qry.all()))
# print(qry.all())

# # Convert the result to a pandas DataFrame
# data = {'id': [1, 2],
#         'symbol': ['ETH-BTC-USDT', 'ETH-BTC-TUSD'],
#         'sub_pairs_list': ['BTCUSDT, ETHBTC, ETHUSDT', 'BTCTUSD, ETHBTC'],
#         'best_bid_prices_list': ['43008.88, 0.05355, 2303.76', '43558.59, 0.05355'],
#         'best_ask_prices_list': ['43008.89, 0.05356, 2303.77', '43558.6, 0.05356']}


result = DataLoader().get_latest_data()
df = pd.DataFrame(result, columns=['id', 'assets', 'sub_pairs_list', 'best_bid_prices_list', 'best_ask_prices_list'])
df[['sub_pair_A', 'sub_pair_B', 'sub_pair_C']] = df['sub_pairs_list'].str.split(', ', expand=True)
df[['bid_price_A', 'bid_price_B', 'bid_price_C']] = df['best_bid_prices_list'].str.split(', ', expand=True).astype(float)

# Split 'best_ask_prices_list' into three columns and convert values to float
df[['ask_price_A', 'ask_price_B', 'ask_price_C']] = df['best_ask_prices_list'].str.split(', ', expand=True).astype(float)
# Display the DataFrame
columns_to_check = ['sub_pair_A', 'sub_pair_B', 'sub_pair_C', 'bid_price_A', 'bid_price_B', 'bid_price_C',
                     'ask_price_A', 'ask_price_B', 'ask_price_C']

# Drop rows where any of the specified columns contain NaN
df = df.dropna(subset=columns_to_check)
pd.set_option('display.max_rows', None)
print(len(df))
display(df)


22


,id,assets,sub_pairs_list,best_bid_prices_list,best_ask_prices_list,sub_pair_A,sub_pair_B,sub_pair_C,bid_price_A,bid_price_B,bid_price_C,ask_price_A,ask_price_B,ask_price_C
0,1,ETH-BTC-USDT,"BTCUSDT, ETHBTC, ETHUSDT","43080.32, 0.05353, 2306.28","43080.33, 0.05354, 2306.29",BTCUSDT,ETHBTC,ETHUSDT,4.308032e+04,5.353000e-02,2306.28000,4.308033e+04,5.354000e-02,2306.29000
1,2,ETH-BTC-TUSD,"BTCTUSD, ETHBTC, ETHTUSD","43609.96, 0.05353, 2334.31","43609.97, 0.05354, 2334.51",BTCTUSD,ETHBTC,ETHTUSD,4.360996e+04,5.353000e-02,2334.31000,4.360997e+04,5.354000e-02,2334.51000
3,4,ETH-BTC-USDC,"BTCUSDC, ETHBTC, ETHUSDC","43067.86, 0.05353, 2305.26","43067.87, 0.05354, 2305.27",BTCUSDC,ETHBTC,ETHUSDC,4.306786e+04,5.353000e-02,2305.26000,4.306787e+04,5.354000e-02,2305.27000
6,7,ETH-BTC-TRY,"BTCTRY, ETHBTC, ETHTRY","1324369.0, 0.05353, 70910.0","1324585.0, 0.05354, 70911.0",BTCTRY,ETHBTC,ETHTRY,1.324369e+06,5.353000e-02,70910.00000,1.324585e+06,5.354000e-02,70911.00000
7,8,ETH-BTC-EUR,"BTCEUR, ETHBTC, ETHEUR","39946.24, 0.05353, 2137.85","39946.25, 0.05354, 2137.89",BTCEUR,ETHBTC,ETHEUR,3.994624e+04,5.353000e-02,2137.85000,3.994625e+04,5.354000e-02,2137.89000
8,9,ETH-BTC-ZAR,"BTCZAR, ETHBTC, ETHZAR","831772.0, 0.05353, 44488.0","831906.0, 0.05354, 44578.0",BTCZAR,ETHBTC,ETHZAR,8.317720e+05,5.353000e-02,44488.00000,8.319060e+05,5.354000e-02,44578.00000
11,12,ETH-BTC-BIDR,"BTCBIDR, ETHBIDR, ETHBTC","676501326.0, 36169199.0, 0.05352","677572784.0, 36288041.0, 0.05353",BTCBIDR,ETHBIDR,ETHBTC,6.765013e+08,3.616920e+07,0.05352,6.775728e+08,3.628804e+07,0.05353
13,14,ETH-BTC-DAI,"BTCDAI, ETHBTC, ETHDAI","43063.71, 0.05352, 2304.65","43066.97, 0.05353, 2304.66",BTCDAI,ETHBTC,ETHDAI,4.306371e+04,5.352000e-02,2304.65000,4.306697e+04,5.353000e-02,2304.66000
15,16,ETH-BTC-BRL,"BTCBRL, ETHBRL, ETHBTC","215746.0, 11546.54, 0.05352","215773.0, 11549.56, 0.05353",BTCBRL,ETHBRL,ETHBTC,2.157460e+05,1.154654e+04,0.05352,2.157730e+05,1.154956e+04,0.05353
16,17,ETH-BTC-UAH,"BTCUAH, ETHBTC, ETHUAH","1917718.0, 0.05352, 102416.0","1920573.0, 0.05353, 102800.0",BTCUAH,ETHBTC,ETHUAH,1.917718e+06,5.352000e-02,102416.00000,1.920573e+06,5.353000e-02,102800.00000
